## DST Assessment 3

In this assessment we are going to attempt topic modelling on some cyber security related data.

In [51]:
import pandas as pd
from urllib.request import urlopen
from urllib.error import HTTPError
import pickle
import requests
import datetime as dt
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import wordnet
import numpy as np
from io import StringIO
import gzip

In [2]:
start = dt.datetime.now()
df = pd.read_csv("https://github.com/Galeforse/DST-Assessment-03/raw/master/Data/master_log_50.csv.gz")
print("Data fetched in:" ,dt.datetime.now()-start)

Data fetched in: 0:00:36.322377


In [3]:
print(len(df))
df.head()

7940261


,anon_ip,log
0,161.166.1.23,Jan 5 03:23:40 161.166.1.23 sshd[27076]: pam_...
1,161.166.1.23,Jan 5 03:23:42 161.166.1.23 sshd[27076]: Fail...
2,161.166.1.23,Jan 5 03:23:54 161.166.1.23 sshd[27076]: Fail...
3,161.166.1.23,Jan 5 03:23:54 161.166.1.23 sshd[27079]: Disc...
4,161.166.1.23,Jan 5 03:24:00 161.166.1.23 sshd[27080]: Fail...


In [4]:
df["log"].iloc[500000]

'Jan 30 08:52:10 localhost sshd[12241]: Disconnecting: Too many authentication failures for XXXXX'

In [5]:
data_text = df[['log']]
data_text["index"] = data_text.index
print(len(data_text))
print(data_text[:5])

7940261
                                                 log  index
0  Jan  5 03:23:40 161.166.1.23 sshd[27076]: pam_...      0
1  Jan  5 03:23:42 161.166.1.23 sshd[27076]: Fail...      1
2  Jan  5 03:23:54 161.166.1.23 sshd[27076]: Fail...      2
3  Jan  5 03:23:54 161.166.1.23 sshd[27079]: Disc...      3
4  Jan  5 03:24:00 161.166.1.23 sshd[27080]: Fail...      4


# Test Regex

In [6]:
import re
def regex(text):
    #text = re.sub('<[ˆ>]*>', '', text)
    #emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    #text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    text = re.sub(r"[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+","",text)
    text = re.sub(r"[0-9]{2}\:[0-9]{2}\:[0-9]{2}","",text)
    text = re.sub(r"XXXXX","",text)
    return text

In [7]:
import re
def regex(text):
    text = re.sub(r"[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+"," ",text)
    text = re.sub(r"[0-9]{2}\:[0-9]{2}\:[0-9]{2}"," ",text)
    text = re.sub(r'\d+'," ",text)
    text = re.sub(r"[^A-Za-z0-9 ]+"," ",text)
    text = re.sub(r"XXXXX"," ",text)
    text = re.sub(r"sshd"," ",text)
    return text

In [8]:
text = "jeremy said: hello, how are you doing?   112.116.2.45   114.117.45.234   25.6.7   01:25:53  17:59:23 password failed for XXXXX on port: 73  ssh=tty  uid="

In [9]:
regex(text)

'jeremy said  hello  how are you doing                         password failed for   on port     ssh tty  uid '

In [10]:
re.sub(r'\d+', '', '123hello56 456world 89')

'hello world '

# Lematizing

In [11]:
lmtzr = nltk.WordNetLemmatizer().lemmatize

def normalize_text(text):
    ## Runs on documents (vector of words)
    word_pos = nltk.pos_tag(nltk.word_tokenize(text))
    lemm_words = [lmtzr(sw[0], get_wordnet_pos(sw[1])) for sw in word_pos]

    return [x.lower() for x in lemm_words]

def preprocess(text):
    ## Runs on documents (vector of words)
    lemmatizer = nltk.WordNetLemmatizer()
    return([lemmatizer.lemmatize(i) for i in text.split()])

In [12]:
en_stop = set(nltk.corpus.stopwords.words('english'))

from nltk.corpus import wordnet as wn
def get_lemma(word):
    ## morphy does a lemma lookup and word standardization
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

## lemmatize
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

## This version is for comparison
def tokenizer(text):
    ## Runs on documents (vector of words)
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

def ssh_preprocessor(text):
    test = regex(text)
    tokenizer(test)    

In [13]:
from gensim import parsing
doc_sample = data_text[data_text['index'] == 16].values[0][0]

print('original document: ')
print(doc_sample)
print('\n simpler tokenized and lemmatized document: ')
print(preprocess(doc_sample))
print('\n method removing stop words: ')
print(tokenizer(doc_sample))
testvar = tokenizer(doc_sample)

original document: 
Jan  5 03:24:56 161.166.1.23 sshd[27097]: Disconnecting: Too many authentication failures for XXXXX

 simpler tokenized and lemmatized document: 
['Jan', '5', '03:24:56', '161.166.1.23', 'sshd[27097]:', 'Disconnecting:', 'Too', 'many', 'authentication', 'failure', 'for', 'XXXXX']

 method removing stop words: 
['03:24:56', '161.166.1.23', 'sshd', '27097', 'Disconnecting', 'many', 'authentication', 'failure', 'XXXXX']


In [14]:
doc_sample = data_text[data_text['index'] == 18].values[0][0]

print('\n Text preprocess')

def preprocess(text):
    complete = tokenizer(regex(text))
    return complete

print(preprocess(doc_sample))


 Text preprocess
['Failed', 'password', 'port']


In [15]:
preprocess(doc_sample)

['Failed', 'password', 'port']

In [16]:
type(testvar[1])

str

In [17]:
processed_docs = data_text['log'][:20].map(preprocess)

In [18]:
data_text["log"][18]

'Jan  5 03:25:08 161.166.1.23 sshd[27098]: Failed password for XXXXX from 5.140.148.58 port 3400 ssh2'

In [19]:
processed_docs[18]

['Failed', 'password', 'port']

In [20]:
len(data_text)

7940261

In [21]:
start=dt.datetime.now()
processed = data_text['log'].map(preprocess)
print("Data Processed in: " + str(dt.datetime.now()-start))

In [22]:
processed

0          [unix, auth, authentication, failure, logname,...
1                                   [Failed, password, port]
2                                   [Failed, password, port]
3             [Disconnecting, many, authentication, failure]
4                                   [Failed, password, port]
                                 ...                        
7940256                                      [Invalid, user]
7940257                                      [Invalid, user]
7940258                                      [Invalid, user]
7940259                                      [Invalid, user]
7940260                                      [Invalid, user]
Name: log, Length: 7940261, dtype: object

# Corpus creation

In [23]:
dictionary = gensim.corpora.Dictionary(processed)

count = 0
for k,v  in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
print(len(dictionary))

0 auth
1 authentication
2 euid
3 failure
4 logname
5 rhost
6 ruser
7 unix
8 user
9 Failed
10 password
1011


In [24]:
dictionary2 = dictionary
dictionary2.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
len(dictionary2)

465

In [25]:
#try:
#    print("Reading corpus from pickle")
#    corpus=pickle.load(open('G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/data/main/corpus.pkl', 'rb'))
#except FileNotFoundError:
#    print("Creating corpus and saving to pickle")
#    corpus = [dictionary2.doc2bow(doc) for doc in processed]
#    pickle.dump(corpus, open('G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/data/main/corpus.pkl', 'wb'))
#    pickle.dump(dictionary2, open('G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/data/main/dictionary.pkl', 'wb'))

In [54]:
try:
    print("Attempting to read corpus from pickle")
    corpus=pickle.load(urlopen('https://github.com/Galeforse/DST-Assessment-03/raw/master/Data/main/corpus_50.pkl'))
    print("Corpus read from pickle")
except HTTPError as err:
    if err.code == 404:
        print("Pickle not found, creating corpus and saving to pickle")
        start=dt.datetime.now()
        corpus = [dictionary2.doc2bow(doc) for doc in processed]
        fp=gzip.open('G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/data/main/corpus_50.pkl.gz','wb')
        pickle.dump(corpus,fp)
        fp.close()        
        #pickle.dump(dictionary2, open('G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-03/data/main/dictionary_50.pkl', 'wb'))
        print("Pickle saved. Time taken: " + str(dt.datetime.now()-start))
    else:
        raise

Attempting to read corpus from pickle
Pickle not found, creating corpus and saving to pickle
Pickle saved. Time taken: 0:01:21.531760


## References

[SSH Dataset from here](https://www.simpleweb.org/wiki/index.php/SSH_datasets)

[Specifically this dataset](http://traces.simpleweb.org/ssh_datasets/dataset2_log_files.tgz)

[SSH Topic Modelling](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7117015)

[G